In [2]:
!pip install fuzzywuzzy
!pip install python-Levenshtein
import pandas as pd
import numpy as np
import sqlite3
from fuzzywuzzy import fuzz

You should consider upgrading via the 'pip install --upgrade pip' command.
     |████████████████████████████████| 51kB 510kB/s eta 0:00:011
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp35-cp35m-linux_x86_64.whl size=152482 sha256=8265472da0f9d89e0c56ab5a141b9ff6b791f48a6ab9bd9422b7e4ad8838453b
  Stored in directory: /home/nbuser/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
#read necessary tables from sqlite database to pd.Dataframes
'''con = sqlite3.connect('european_soccer_data.sqlite')
match_table = pd.read_sql_query('SELECT * from Match', con)
team_table = pd.read_sql_query('SELECT * from Team', con)
country_table = pd.read_sql_query('SELECT * from Country', con)
con.close()'''

#write to csv to make future edits easier to start
'''match_csv = match_table.to_csv('match.csv', index=False)
team_csv = team_table.to_csv('team.csv', index=False)
country_csv = country_table.to_csv('country.csv', index=False)'''

#read csv to pd.Dataframe
match_df = pd.read_csv('match.csv')
team_df = pd.read_csv('team.csv')
country_df = pd.read_csv('country.csv')

In [4]:
country_df.head()

,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy


In [5]:
team_df.head()

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB


In [6]:
match_df.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


In [7]:
#manual league name dict for leagues of interest, small dataset so manual works
league_name_dict = {'England':'Premier League', 'France':'Ligue 1', 'Germany':'1 Bundesliga', 'Italy':'Serie A',
                    'Netherlands':'Eredivisie', 'Portugal':'Liga Nos', 'Spain':'Primera Division'}

#add league name column to country_df
country_df['league_name'] = ''
for i, row in country_df.iterrows():
    league_name = 'NA'
    if row['name'] in league_name_dict.keys():
        league_name = league_name_dict[row['name']]
    country_df.at[i, 'league_name'] = league_name

#subset country_df to leagues I have match data for
country_df = country_df[country_df.league_name != 'NA'].reset_index(drop=True)
country_df

,id,name,league_name
0,1729,England,Premier League
1,4769,France,Ligue 1
2,7809,Germany,1 Bundesliga
3,10257,Italy,Serie A
4,13274,Netherlands,Eredivisie
5,17642,Portugal,Liga Nos
6,21518,Spain,Primera Division


In [8]:
#create dict of {team id: team name} and {country id: league name} which will be used to create new columns in match_df
team_id_dict = pd.Series(team_df.team_long_name.values, index=team_df.team_api_id).to_dict()
country_id_dict = pd.Series(country_df.league_name.values, index=country_df.id).to_dict()

In [9]:
#add empty column to specific index in match_df, can't run this code multiple times, comment out
match_df.insert(7, 'home_team_name', ['' for i in range(match_df.shape[0])])
match_df.insert(8, 'away_team_name', ['' for i in range(match_df.shape[0])])
match_df.insert(3, 'league_name', ['' for i in range(match_df.shape[0])])

for i, row in match_df.iterrows():
    league_name = 'NA'
    if row['country_id'] in country_id_dict.keys():
        league_name = country_id_dict[row['country_id']]
    match_df.at[i, 'league_name'] = league_name

#subset match_df to leagues I have transfer data for
match_df = match_df[match_df.league_name != 'NA'].reset_index(drop=True)

for i, row in match_df.iterrows():
    match_df.at[i, 'home_team_name'] = team_id_dict[row['home_team_api_id']]
    match_df.at[i, 'away_team_name'] = team_id_dict[row['away_team_api_id']]

match_df.head()

,id,country_id,league_id,league_name,season,stage,date,match_api_id,home_team_name,away_team_name,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1729,1729,1729,Premier League,2008/2009,1,2008-08-17 00:00:00,489042,Manchester United,Newcastle United,...,10.00,1.28,5.5,12.00,1.30,4.75,10.0,1.29,4.50,11.00
1,1730,1729,1729,Premier League,2008/2009,1,2008-08-16 00:00:00,489043,Arsenal FC,West Bromwich Albion,...,12.00,1.25,6.0,13.00,1.22,5.50,13.0,1.22,5.00,13.00
2,1731,1729,1729,Premier League,2008/2009,1,2008-08-16 00:00:00,489044,Sunderland AFC,Liverpool FC,...,1.73,5.50,3.8,1.65,5.00,3.40,1.7,4.50,3.40,1.73
3,1732,1729,1729,Premier League,2008/2009,1,2008-08-16 00:00:00,489045,West Ham United,Wigan Athletic,...,3.75,1.90,3.5,4.35,1.91,3.25,4.0,1.91,3.25,3.80
4,1733,1729,1729,Premier League,2008/2009,1,2008-08-17 00:00:00,489046,Aston Villa,Manchester City,...,3.75,1.90,3.5,4.35,1.91,3.25,4.0,1.91,3.30,3.75


In [10]:
#removing some unnecessary columns, can only run once, comment out

for i in range(1,12):
    match_df = match_df.drop(['home_player_X'+str(i),'away_player_X'+str(i),'home_player_Y'+str(i),
                              'away_player_Y'+str(i),'home_player_'+str(i),'away_player_'+str(i)], axis=1)

for column in match_df.columns[-30:]:
    match_df = match_df.drop(column, axis=1)

match_df = match_df.drop(['country_id', 'id'], axis=1)

print(match_df.shape)
print([column for column in match_df.columns])
match_df.head()

(19085, 20)
['league_id', 'league_name', 'season', 'stage', 'date', 'match_api_id', 'home_team_name', 'away_team_name', 'home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal', 'goal', 'shoton', 'shotoff', 'foulcommit', 'card', 'cross', 'corner', 'possession']


,league_id,league_name,season,stage,date,match_api_id,home_team_name,away_team_name,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,goal,shoton,shotoff,foulcommit,card,cross,corner,possession
0,1729,Premier League,2008/2009,1,2008-08-17 00:00:00,489042,Manchester United,Newcastle United,10260,10261,1,1,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>56</comment><event...
1,1729,Premier League,2008/2009,1,2008-08-16 00:00:00,489043,Arsenal FC,West Bromwich Albion,9825,8659,1,0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card />,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>65</comment><event...
2,1729,Premier League,2008/2009,1,2008-08-16 00:00:00,489044,Sunderland AFC,Liverpool FC,8472,8650,0,1,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>45</comment><event...
3,1729,Premier League,2008/2009,1,2008-08-16 00:00:00,489045,West Ham United,Wigan Athletic,8654,8528,2,1,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>50</comment><event...
4,1729,Premier League,2008/2009,1,2008-08-17 00:00:00,489046,Aston Villa,Manchester City,10252,8456,4,2,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><corners>1</corners></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>51</comment><event...


In [11]:
#edit season column to int of start year rather than string of years, ex. 2008 refers to 2008/2009 season
match_df.season = match_df.season.apply(lambda x : int(x[:4]))
match_df.head()

,league_id,league_name,season,stage,date,match_api_id,home_team_name,away_team_name,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,goal,shoton,shotoff,foulcommit,card,cross,corner,possession
0,1729,Premier League,2008,1,2008-08-17 00:00:00,489042,Manchester United,Newcastle United,10260,10261,1,1,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>56</comment><event...
1,1729,Premier League,2008,1,2008-08-16 00:00:00,489043,Arsenal FC,West Bromwich Albion,9825,8659,1,0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card />,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>65</comment><event...
2,1729,Premier League,2008,1,2008-08-16 00:00:00,489044,Sunderland AFC,Liverpool FC,8472,8650,0,1,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>45</comment><event...
3,1729,Premier League,2008,1,2008-08-16 00:00:00,489045,West Ham United,Wigan Athletic,8654,8528,2,1,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>50</comment><event...
4,1729,Premier League,2008,1,2008-08-17 00:00:00,489046,Aston Villa,Manchester City,10252,8456,4,2,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><corners>1</corners></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>51</comment><event...


In [12]:
#add home team win, loss, draw columns
match_df.insert(10, 'home_win', [False for i in range(match_df.shape[0])])
match_df.insert(11, 'away_win', [False for i in range(match_df.shape[0])])
match_df.insert(12, 'draw', [False for i in range(match_df.shape[0])])

In [13]:
#loop through rows, update win, loss, draw columns to True where necessary
for i, row in match_df.iterrows():
    if row['home_team_goal'] > row['away_team_goal']:
        match_df.at[i, 'home_win'] = True
    elif row['home_team_goal'] < row['away_team_goal']:
        match_df.at[i, 'away_win'] = True
    else:
        match_df.at[i, 'draw'] = True

match_df[['home_team_goal', 'away_team_goal', 'home_win', 'away_win', 'draw']].head()

,home_team_goal,away_team_goal,home_win,away_win,draw
0,1,1,False,False,True
1,1,0,True,False,False
2,0,1,False,True,False
3,2,1,True,False,False
4,4,2,True,False,False


In [14]:
#sort match_df by league, season, stage
match_df = match_df.groupby(['season', 'league_name']).apply(lambda x: x.sort_values(['stage'])).reset_index(drop=True)
match_df.head(50)

,league_id,league_name,season,stage,date,match_api_id,home_team_name,away_team_name,home_team_api_id,away_team_api_id,...,home_team_goal,away_team_goal,goal,shoton,shotoff,foulcommit,card,cross,corner,possession
0,7809,1 Bundesliga,2008,1,2008-08-15 00:00:00,499317,Bayern Munich,Hamburger SV,9823,9790,...,2,2,<goal><value><comment>n</comment><stats><goals...,<shoton />,<shotoff />,<foulcommit />,<card><value><comment>y</comment><stats><ycard...,<cross />,<corner />,<possession />
1,7809,1 Bundesliga,2008,1,2008-08-17 00:00:00,499324,Borussia Mönchengladbach,VfB Stuttgart,9788,10269,...,1,3,<goal><value><comment>n</comment><stats><goals...,<shoton />,<shotoff />,<foulcommit />,<card><value><comment>y</comment><stats><ycard...,<cross />,<corner />,<possession />
2,7809,1 Bundesliga,2008,1,2008-08-16 00:00:00,499323,FC Energie Cottbus,TSG 1899 Hoffenheim,8398,8226,...,0,3,<goal><value><comment>n</comment><stats><goals...,<shoton />,<shotoff />,<foulcommit />,<card><value><comment>y</comment><stats><ycard...,<cross />,<corner />,<possession />
3,7809,1 Bundesliga,2008,1,2008-08-16 00:00:00,499322,DSC Arminia Bielefeld,SV Werder Bremen,9912,8697,...,2,2,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession />
4,7809,1 Bundesliga,2008,1,2008-08-16 00:00:00,499325,Karlsruher SC,VfL Bochum,8295,9911,...,1,0,<goal><value><comment>n</comment><stats><goals...,<shoton />,<shotoff />,<foulcommit />,<card><value><comment>y</comment><stats><ycard...,<cross />,<corner />,<possession />
5,7809,1 Bundesliga,2008,1,2008-08-16 00:00:00,499320,VfL Wolfsburg,1. FC Köln,8721,8722,...,2,1,<goal><value><comment>n</comment><stats><goals...,<shoton />,<shotoff />,<foulcommit />,<card><value><comment>y</comment><stats><ycard...,<cross />,<corner />,<possession />
6,7809,1 Bundesliga,2008,1,2008-08-16 00:00:00,499319,FC Schalke 04,Hannover 96,10189,9904,...,3,0,<goal><value><comment>n</comment><stats><goals...,<shoton />,<shotoff />,<foulcommit />,<card><value><comment>y</comment><stats><ycard...,<cross />,<corner />,<possession />
7,7809,1 Bundesliga,2008,1,2008-08-16 00:00:00,499318,Bayer 04 Leverkusen,Borussia Dortmund,8178,9789,...,2,3,<goal><value><comment>n</comment><stats><goals...,<shoton />,<shotoff />,<foulcommit />,<card><value><comment>y</comment><stats><ycard...,<cross />,<corner />,<possession />
8,7809,1 Bundesliga,2008,1,2008-08-17 00:00:00,499321,Eintracht Frankfurt,Hertha BSC,9810,8177,...,0,2,<goal><value><comment>n</comment><stats><goals...,<shoton />,<shotoff />,<foulcommit />,<card><value><comment>y</comment><stats><ycard...,<cross />,<corner />,<possession />
9,7809,1 Bundesliga,2008,2,2008-08-23 00:00:00,499334,TSG 1899 Hoffenheim,Borussia Mönchengladbach,8226,9788,...,1,0,<goal><value><comment>n</comment><stats><goals...,<shoton />,<shotoff />,<foulcommit />,<card><value><comment>y</comment><stats><ycard...,<cross />,<corner />,<possession />


In [55]:
#match_clean_csv = match_df.to_csv('match_clean.csv')
match_df_clean = pd.read_csv('match_clean.csv')

In [58]:
def get_league_table(season, league_name): # returns dataframe of end of season table for given year, league
    season_df = match_df_clean[(match_df_clean['league_name']==league_name) & (match_df_clean['season']==season)] #subset match_df to specified season, league
    
    league_table_df = pd.DataFrame(0, columns=['Wins', 'Draws', 'Losses', 'Goals_For', 'Goals_Against', 'Goal_Difference', 'Points'],
                                   index=season_df['home_team_name'].unique()).rename_axis('Team') #initialize every column to zero, team names as indices
        
    for i, row in season_df.iterrows(): #update goals for/against, win/loss/draw columns
        league_table_df.loc[row['home_team_name'], 'Goals_For'] += row['home_team_goal']
        league_table_df.loc[row['home_team_name'], 'Goals_Against'] += row['away_team_goal']
        league_table_df.loc[row['away_team_name'], 'Goals_For'] += row['away_team_goal']
        league_table_df.loc[row['away_team_name'], 'Goals_Against'] += row['home_team_goal']
        
        if row['home_win'] == True:
            league_table_df.loc[row['home_team_name'], 'Wins'] += 1
            league_table_df.loc[row['away_team_name'], 'Losses'] += 1
        elif row['away_win'] == True:
            league_table_df.loc[row['home_team_name'], 'Losses'] += 1
            league_table_df.loc[row['away_team_name'], 'Wins'] += 1
        else:
            league_table_df.loc[row['home_team_name'], 'Draws'] += 1
            league_table_df.loc[row['away_team_name'], 'Draws'] += 1
        
    league_table_df['Goal_Difference'] = league_table_df['Goals_For'] - league_table_df['Goals_Against']
    league_table_df['Points'] = league_table_df['Wins']*3 + league_table_df['Draws']
    league_table_df = league_table_df.sort_values(['Points', 'Goal_Difference'], ascending=False).reset_index()
    league_table_df.index = pd.RangeIndex(start=1, stop=len(league_table_df)+1, step=1)
    
    return league_table_df

get_league_table(2009, 'Serie A')

,Team,Wins,Draws,Losses,Goals_For,Goals_Against,Goal_Difference,Points
1,Inter Milan,24,10,4,75,34,41,82
2,AS Roma,24,8,6,68,41,27,80
3,AC Milan,20,10,8,60,39,21,70
4,UC Sampdoria,19,10,9,49,41,8,67
5,US Palermo,18,11,9,59,47,12,65
6,SSC Napoli,15,14,9,50,43,7,59
7,Juventus FC,16,7,15,55,56,-1,55
8,Parma FC,14,10,14,46,51,-5,52
9,Genoa CFC,14,9,15,57,61,-4,51
10,Bari,13,11,14,49,49,0,50


In [90]:
filenames = ['dutch_eredivisie.csv', 'english_premier_league.csv', 'french_ligue_1.csv', 'german_bundesliga_1.csv',
                'italian_serie_a.csv', 'portugese_liga_nos.csv', 'spanish_primera_division.csv']

transfers_df_list = []

for i in range(2007, 2017):
    for file in filenames:
        df = pd.read_csv('transfer_data/'+str(i)+'/'+file)
        df.insert(0, 'club_api_id', '')
        transfers_df_list.append(df)

transfers_df = pd.concat(transfers_df_list)

transfers_df.head()

,club_api_id,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,fee_cleaned,league_name,year,season
0,,PSV Eindhoven,Danny Koevermans,28.0,Centre-Forward,AZ Alkmaar,£6.21m,in,6.21,Eredivisie,2007,2007/2008
1,,PSV Eindhoven,Danko Lazovic,24.0,Second Striker,Vitesse,£5.40m,in,5.40,Eredivisie,2007,2007/2008
2,,PSV Eindhoven,Balázs Dzsudzsák,21.0,Left Winger,Debrecen,£2.25m,in,2.25,Eredivisie,2007,2007/2008
3,,PSV Eindhoven,Cássio,20.0,Goalkeeper,Grêmio,£900k,in,0.90,Eredivisie,2007,2007/2008
4,,PSV Eindhoven,Kenneth Pérez,32.0,Centre-Forward,Ajax,£900k,in,0.90,Eredivisie,2007,2007/2008


In [150]:
rev_team_id_dict = {value:key for key, value in team_id_dict.items()}



'''
for i, row in transfers_df.iterrows():
    if row['club_name'] in list(team_df.team_long_name):
        transfers_df.set_value(i, 'club_api_id', int(team_df.team_api_id[team_df['team_long_name']==row['club_name']]))
    else:
        transfers_df.set_value(i, 'club_api_id', 'NA')
'''

"for i, row in transfers_df.iterrows():\n    api_id = 'NA'\n    if row['club_name'] in rev_team_id_dict.keys():\n        api_id = rev_team_id_dict[row['club_name']]\n    transfers_df.at[i, 'club_api_id'] = api_id"

In [151]:
transfers_df.head()

,club_api_id,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,fee_cleaned,league_name,year,season
0,8634,PSV Eindhoven,Danny Koevermans,28.0,Centre-Forward,AZ Alkmaar,£6.21m,in,6.21,Eredivisie,2007,2007/2008
1,8634,PSV Eindhoven,Danko Lazovic,24.0,Second Striker,Vitesse,£5.40m,in,5.40,Eredivisie,2007,2007/2008
2,8634,PSV Eindhoven,Balázs Dzsudzsák,21.0,Left Winger,Debrecen,£2.25m,in,2.25,Eredivisie,2007,2007/2008
3,8634,PSV Eindhoven,Cássio,20.0,Goalkeeper,Grêmio,£900k,in,0.90,Eredivisie,2007,2007/2008
4,8634,PSV Eindhoven,Kenneth Pérez,32.0,Centre-Forward,Ajax,£900k,in,0.90,Eredivisie,2007,2007/2008


In [ ]:
transfers_df['fee_cleaned'] = transfers_df['fee_cleaned'].apply(lambda x: x*1000000) #fee originally in millions
transfers_in = transfers_df[transfers_df['transfer_movement'] == 'in'] #subset purchases
transfers_out = transfers_df[transfers_df['transfer_movement'] == 'out'] #subset sales

transfers_in.head()

In [57]:
#money spent per club per year
transfers_by_year = transfers_in.groupby(['club_name', 'year']).sum().drop('age', axis=1)


#total spent by club between 2007-2016
total_spent = transfers_by_year.groupby(level=0).sum().sort_values('fee_cleaned', ascending=False)

#top 100 spending clubs during period of interest
total_spent.head(100)

top_amounts_spent = transfers_by_year.groupby(level=[0,1]).sum().sort_values('fee_cleaned', ascending=False).head(100)